In [ ]:
%pip install unstructured==0.7.12
%pip install langchain
%pip install openai
%pip install chromadb
%pip install tiktoken

In [ ]:

import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.chains.llm import LLMChain
from langchain.memory import ConversationBufferMemory, VectorStoreRetrieverMemory
from dotenv import load_dotenv

load_dotenv() 

openai_api_key = os.getenv("OPENAI_API_KEY")

Chat Module is the conversational pipeline that tracks chosen steps in proof and selects the next step to be proved. It will use a router chain to add the choice to the context module to provide more recent context in the process. 

It has weaker memory of the pdf, but can refer to context module for other references

Might not need chat module since we can just sent proof decision to context module directly. 

In [ ]:
loader = DirectoryLoader('../doc/', glob='linear_map.pdf')
documents = loader.load()

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
# Create embeddings
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
# load it into Chroma
vectorstore = Chroma.from_documents(texts, embeddings)

# qa = VectorDBQA.from_chain_type(llm=OpenAI(openai_api_key = openai_api_key), chain_type="stuff", vectorstore=vectorstore)


In [ ]:
retriever = vectorstore.as_retriever()
# For conversation asking at a specific step in proof
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True, retriever=retriever)

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a chatbot having a conversation with a human."), # The persistent system prompt
    MessagesPlaceholder(variable_name="chat_history"), # Where the memory will be stored.
    HumanMessagePromptTemplate.from_template("{human_input}"), # Where the human input will injectd
])

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k", openai_api_key=openai_api_key)
# get context of the past proof steps
llm_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

In [ ]:
query = "what is my name"
llm_chain.run(query)